# costPower

In [51]:
import requests
import pymongo
#/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/8/1124193D-09F5-4711-AB9A-01848E3B88E4/8/?fileType=csv
res=requests.get("https://sip.einvoice.nat.gov.tw/ods-main/ODS308E/download/691C0280-CEFB-488F-9E71-6AA4F39A41CD/8/1124193D-09F5-4711-AB9A-01848E3B88E4/8/?fileType=csv")
x=res.text
keys=x.replace('"',"").replace("\u3000","").replace("\U000fffb4","").replace("\r","").split("\n")[0].split(",")
values=[data.replace("\u3000","")
        .replace("\U000fffb4","")
        .replace("\U000fb56f","塭")
        .replace("\r","")
        .split(",") for data in x.replace('"',"").split("\n")[1:]]
datalist=[]
for valueList in values:
    data={}
    for key,value in zip(keys,valueList):
        data[key]=value
    datalist.append(data)
dataFinal=[]
for i in datalist:
    data={}
    if i.get('主行業別',"")=='餐飲業' and i.get('年度',"")=='2017' and i.get("消費熱度計算來源","")=="電子發票":
        data['add']=i['縣市']+i['鄉鎮市區']+i['村里']
        data['bigadd']=i['縣市'].replace("臺","台")
        data['smalladd']=i['鄉鎮市區']
        data['costPower']=i['銷售額指標']
        dataFinal.append(data)
        
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in dataFinal:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
costPower=[add for add in dataFinal if 'lat' in add]
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.CostPower
try:
    collection.drop()
    collection.insert_many(costPower,ordered=False)
except:
    print("some data has same _id")
    
print("xxx")
client.close()

['MRTinfo', 'Watsons', 'pxmart', 'busData', 'HRdata104', 'info3Store', 'wowprimediendata', 'info591', 'departmentStore', 'websites591', 'carrefour', 'taiwanInfo', 'smallStyleCount', 'addressCoordinate', 'Nhuman', 'taiwanInfoStoneTwo', 'ipeenWebsite', 'taiwanInfoHot7', 'bigStyleCount', 'ipeenInfo', 'conStore', 'testDB', 'CostPower', 'infoClinic']
xxx


# humanDensity

In [23]:
import requests
import json
import pymongo
import re
# res=requests.get("https://quality.data.gov.tw/dq_download_json.php?nid=32973&md5_url=52629fae1f1fed4ef81fcf706878dd80")
# x=json.loads(res.text)

res=requests.get("http://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=4C79B290-D72F-4DC9-A919-9023F141E32C")
datas=res.text.split("\r\n")
titles=datas[0].split(",")
x=[{title:value for title,value in zip(titles,data.split(","))} for data in datas[2:]]

data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五").replace("?","館"),
       'Nhuman':add['people_total']}for add in x[1:] if "site_id" in add]
for i in data:
    try:
        i["bigadd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                               i["add"])[0][0].replace("臺","台")
    except:
        i['bigadd']=0
    try:
        i["smalladd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                           i["add"])[0][1]
    except:
        i["smalladd"]=0


        
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate
Coordinate={add['add']:{"lat":add['lat'],'lng':add['lng']} for add in list(collection.find({}))}
client.close()

for add in data:
    try:
        add['lat']=Coordinate[add['add']]['lat']
        add['lng']=Coordinate[add['add']]['lng']
        add["_id"]=add.pop('add')
    except:
#         print(add['add'])
        pass
Nhuman=[add for add in data if 'lat' in add]
client=pymongo.mongo_client.MongoClient("localhost",27017,username='j122085',password='850605')
db=client.rawData
collection=db.Nhuman
try:
    collection.drop()
    collection.insert_many(Nhuman,ordered=False)
except:
    print("some data has same _id")
client.close()

['MRTinfo', 'Watsons', 'pxmart', 'busData', 'HRdata104', 'info3Store', 'wowprimediendata', 'info591', 'departmentStore', 'websites591', 'carrefour', 'taiwanInfo', 'smallStyleCount', 'addressCoordinate', 'Nhuman', 'taiwanInfoStoneTwo', 'ipeenWebsite', 'taiwanInfoHot7', 'bigStyleCount', 'ipeenInfo', 'conStore', 'testDB', 'CostPower', 'infoClinic']


# 轉換區里成經緯度

In [ ]:
import requests
import geocoder
import time

res=requests.get("http://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=4C79B290-D72F-4DC9-A919-9023F141E32C")
datas=res.text.split("\r\n")
titles=datas[0].split(",")
x=[{title:value for title,value in zip(titles,data.split(","))} for data in datas[2:]]

data=[{"add":add['site_id'].replace("　","").replace("\U000fffb5","五")+add['village'].replace(" ","").replace("\U000fffb5","五").replace("?","館"),
       'Nhuman':add['people_total']}for add in x[1:] if "site_id" in add]
addlist=[{"add":i['add']} for i in data]


for i in addlist:
    n=0
    while 'lat' not in i:
        try:
            n+=1
            if n>10:
                break
            i['lat'],i['lng']=geocoder.arcgis(i['add']).latlng
            time.sleep(0.5)
            print(i['add'],i['lat'],i['lng'])
        except:
            time.sleep(0.3)
        


        

In [ ]:
import pymongo
client=pymongo.mongo_client.MongoClient("192.168.1.113",27017,username='j122085',password='850605')
# print(client.database_names())
db=client.rawData
print(db.collection_names())
collection=db.addressCoordinate

from pymongo import UpdateOne
adds=[i.pop("add") for i in  addlist]
operations=[UpdateOne({"add":addn},{"$set":data},upsert=True) for addn,data in zip(adds,addlist)]
try:
    collection.bulk_write(operations,ordered=False)
except:
    print("someData has exit")